# Day 5

In [1]:
import pandas as pd
import numpy as np
import re

In [85]:
seats_file = open('input',mode='r')

row=[]
col=[]
for entry in [[seat[:7],seat[-3:]] for seat in seats_file.read().split('\n')]:
    row.append(entry[0])
    col.append(entry[1])
    
seats=pd.DataFrame({'row':row, 'col':col})
seats

,row,col
0,FBBFFBB,LLL
1,FFBFFFB,RLL
2,FFBBBBF,RRL
3,FBFBBBB,RLL
4,BFBBBBF,LLR
...,...,...
841,BFBBFFF,RRL
842,FBFFBBB,LRL
843,FBFBBFF,LRR
844,FBFFBBF,RRL


In [81]:
def lower(vals):
    return[vals[0], vals[0]+(vals[1]-vals[0])//2]
def upper(vals):
    return[(vals[0]+vals[1])//2, vals[1]]

calc_switch = {
    'F': lower,
    'B': upper,
    'L': lower,
    'R': upper
}
def calc_row(row, seat):
    if seat=="" or len(row)==1:
        return row[1]
    else:
        [start, end] = row
        next_row = calc_switch[seat[:1]](row)
        return calc_row(next_row, seat[1:])

## Part 1

### row
**7** entries: range **0** to **127**
 - **F** : lower half (*front*)
 - **B** : upper half (*back*)

### column
**3** entries: range **0** to **7**
 - **L** : lower half
 - **R** : upper half

### seatId
multiply the row by 8, then add the column. In this example, the seat has ID `row * 8 + col = seatId`.


examples
- `BFFFBBFRRR`: row **70**, column **7**, seat ID **567**.
- `FFFBBBFRRR`: row **14**, column **7**, seat ID **119**.
- `BBFFBBFRLL`: row **102**, column **4**, seat ID **820**.

In [82]:
dummy_str="""BFFFBBFRRR
FFFBBBFRRR
BBFFBBFRLL
FBFBBFFRLR"""

row=[]
col=[]
for entry in [[seat[:7],seat[-3:]] for seat in dummy_str.split('\n')]:
    row.append(entry[0])
    col.append(entry[1])
    
dummy=pd.DataFrame({'row':row, 'col':col, 'row_actual':[70,14,102,44],'col_actual':[7,7,4,5],'seat_actual':[567,119,820,357]})

dummy

,row,col,row_actual,col_actual,seat_actual
0,BFFFBBF,RRR,70,7,567
1,FFFBBBF,RRR,14,7,119
2,BBFFBBF,RLL,102,4,820
3,FBFBBFF,RLR,44,5,357


In [83]:
def run_calc(row):
    row['row_calc'] = calc_row([0,127],row['row'])
    row['col_calc'] = calc_row([0,7],row['col'])
    row['seat_calc'] = row['row_calc']*8+row['col_calc']
    return row

dummy.apply(run_calc, axis=1)

,row,col,row_actual,col_actual,seat_actual,row_calc,col_calc,seat_calc
0,BFFFBBF,RRR,70,7,567,70,7,567
1,FFFBBBF,RRR,14,7,119,14,7,119
2,BBFFBBF,RLL,102,4,820,102,4,820
3,FBFBBFF,RLR,44,5,357,44,5,357


In [87]:
res=seats.apply(run_calc, axis=1)

In [89]:
res.seat_calc.max()

858

## Part 2

In [106]:
sorted = res.sort_values(by=['seat_calc'])
sorted[(sorted['seat_calc']!= sorted['seat_calc'].min()) & (sorted.seat_calc - sorted.seat_calc.shift(1) != 1.0)].seat_calc -1

98    557
Name: seat_calc, dtype: int64